<a href="https://colab.research.google.com/github/evanzs/curso-spark/blob/master/spark_ml_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [ ]:

spark_home ='/content/drive/MyDrive/Colab Notebooks/spark-3.5.1-bin-hadoop3.tgz'

# instalar as dependências
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

!tar xf '/content/drive/MyDrive/Colab Notebooks/spark-3.5.1-bin-hadoop3.tgz'
!pip install -q findspark

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.5.1-bin-hadoop3')


from pyspark.sql import SparkSession

In [34]:
## Criando RDD
spark = SparkSession.builder.appName("ml").getOrCreate()




**Trabalhando ML  usando**
- Regressão Linear da LIB DE ML
- Regressão RandomForest que usa arvore de Decisão

In [35]:
## importando classes de ml
from pyspark.ml.regression import LinearRegression,RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler


In [36]:
## importando Dados
import pandas as pd
df  = pd.read_csv("https://raw.githubusercontent.com/evanzs/curso-spark/master/datasets/Carros.csv",sep=';')
df_carros = spark.createDataFrame(df)
df_carros.show(5)


+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



## Vetorizando 3 colunas

- Vetorizado varaiveis cilindros,cilindradas e consumo, transformando essa 3 colunas em apenas uma,reduzindo o tamnho.

- usaremos apenas essa coluna resultado e HP para regressão


In [37]:
carros_temp = df_carros.select('Cilindros','Cilindradas','HP','Consumo')

cols = [("Consumo"),("Cilindradas"),("Cilindros")]
vec_caracteristicas = VectorAssembler(inputCols=cols,outputCol="caracteristicas")
dfv_carros = vec_caracteristicas.transform(carros_temp)
dfv_carros.show(5)

## coluna caracteristicas é a combinaão vetorizada das colunas cilindros,cilindradas e Consumo


+---------+-----------+---+-------+-----------------+
|Cilindros|Cilindradas| HP|Consumo|  caracteristicas|
+---------+-----------+---+-------+-----------------+
|        6|        160|110|     21| [21.0,160.0,6.0]|
|        6|        160|110|     21| [21.0,160.0,6.0]|
|        4|        108| 93|    228|[228.0,108.0,4.0]|
|        6|        258|110|    214|[214.0,258.0,6.0]|
|        8|        360|175|    187|[187.0,360.0,8.0]|
+---------+-----------+---+-------+-----------------+
only showing top 5 rows



In [40]:
## separando dataframe entre teste e treino em 70% treino - 30%  teste
splitConfig = [0.7,0.3]
carros_treino,carros_teste = dfv_carros.randomSplit(splitConfig)
print("quantidade de itens para treino: ",carros_treino.count());
print("quantidades de itens para teste: ",carros_teste.count())

#OBS: o numero é muito pouco mas não iremos validar isso inicialmente

quantidade de itens para treino:  19
quantidades de itens para teste:  13


# Criando modelo REGRESSAO LINEAR
Criamos um modulo de regressão lienar, onde a coluna objetivo seria "HP" e as outras seria a coluna "caracteristica".

In [41]:
## criando medelo
regLinear = LinearRegression(featuresCol="caracteristicas",labelCol="HP");

## usando .fit() para treinar o modulo
model_regLinear = regLinear.fit(carros_treino)


#realizando a previsão do modulo usando o df de teste
result = model_regLinear.transform(carros_teste)
result.show() ##coluna "prediction" seria o resultado

+---------+-----------+---+-------+------------------+------------------+
|Cilindros|Cilindradas| HP|Consumo|   caracteristicas|        prediction|
+---------+-----------+---+-------+------------------+------------------+
|        4|        108| 93|    228| [228.0,108.0,4.0]| 82.20474895782729|
|        4|       1467| 62|    244|[244.0,1467.0,4.0]| 75.51873902507842|
|        6|        160|110|     21|  [21.0,160.0,6.0]|133.96258189772334|
|        6|        225|105|    181| [181.0,225.0,6.0]| 139.6374009909798|
|        6|        258|110|    214| [214.0,258.0,6.0]|140.70275065019825|
|        8|        360|175|    187| [187.0,360.0,8.0]|198.96877503218752|
|        8|        460|215|    104| [104.0,460.0,8.0]|195.30782557598405|
|        4|        121|109|    214| [214.0,121.0,4.0]| 81.60798075016272|
|        4|        757| 52|    304| [304.0,757.0,4.0]| 81.58526795697085|
|        4|       1203| 91|     26| [26.0,1203.0,4.0]|  68.7276737776469|
|        6|        145|175|    197| [1

##**Medindo o nivel de acuracia do modelo feito.**
- utilizando o "RegressionEvaluator" para medir a performace do module criado, passando a metrica desejada.

# Coeficiente de determinação (R2)
- O coeficiente de determinação, também chamado de R², é uma medida de ajuste de um modelo estatístico linear generalizado, como a regressão linear simples ou múltipla, aos valores observados de uma variável aleatória. O R² varia entre 0 e 1, por vezes sendo expresso em termos percentuais

- Quando R2 é igual a 0, isso significa que o modelo não explica nada da variação dos dados, enquanto um R2 igual a 1 indica que o modelo explica toda a variação dos dados. Em outras palavras, o R2 representa a porcentagem da variação nos dados que é explicada pelo modelo.


# Raiz quadrada do erro-médio
-  O Erro Quadrático Médio (RMSE) é o desvio padrão dos valores residuais (erros de previsão). Os valores residuais são uma medida da distância entre a linha de regressão e os pontos de dados. O RMSE é uma medida da dispersão desses valores residuais.


In [43]:
rmse = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse').evaluate(result)
r2 = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='r2').evaluate(result)
mae = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='mae').evaluate(result)

print("resultado de RMSE: ",rmse) ## quanto menor melhor
print("resultado de MAE: ",mae) ##
print("resultado de R2: ",r2) ## quanto mais prox. de 1 melhor


resultado de RMSE:  47.879246290697466
resultado de MAE:  35.466075216649
resultado de R2:  0.6241887032473652


# Criando modelo RandomForest
Criamos um modulo de RandomForest, onde a coluna objetivo seria "HP" e as outras seria a coluna "caracteristica" ( já criado).

In [47]:
# criando modelo
reg_randomForest = RandomForestRegressor(featuresCol="caracteristicas",labelCol="HP")

#treinando modelo
model_randomForest = reg_randomForest.fit(carros_treino)

result_random = model_randomForest.transform(carros_teste)
result_random.show(5)


rmse_random = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse').evaluate(result_random)
r2_random = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='r2').evaluate(result_random)
mae_random = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='mae').evaluate(result_random)


print("resultado de RMSE: ",rmse)
print("resultado de MAE: ",mae)
print("resultado de R2: ",r2)


print("\n resultado de RMSE: ",rmse_random) ## quanto menor melhor
print("resultado de MAE: ",mae_random) ##
print("resultado de R2: ",r2_random) ## quanto mais prox. de 1 melho

+---------+-----------+---+-------+------------------+------------------+
|Cilindros|Cilindradas| HP|Consumo|   caracteristicas|        prediction|
+---------+-----------+---+-------+------------------+------------------+
|        4|        108| 93|    228| [228.0,108.0,4.0]|           93.1875|
|        4|       1467| 62|    244|[244.0,1467.0,4.0]| 97.52000000000001|
|        6|        160|110|     21|  [21.0,160.0,6.0]|126.80803571428571|
|        6|        225|105|    181| [181.0,225.0,6.0]|             118.2|
|        6|        258|110|    214| [214.0,258.0,6.0]|          108.4375|
+---------+-----------+---+-------+------------------+------------------+
only showing top 5 rows

resultado de RMSE:  45.039030228626686
resultado de MAE:  30.118195970695965
resultado de R2:  0.6674528103683037
resultado de RMSE:  45.039030228626686
resultado de MAE:  30.118195970695965
resultado de R2:  0.6674528103683037
